In [19]:

import pandas as pd
from pandas import DataFrame
pd.set_option('display.max_rows', 3000)

import xlrd
xlrd.xlsx.ensure_elementtree_imported(False, None)
xlrd.xlsx.Element_has_iter = True

import openpyxl

import os


In [20]:
def load_data(file_name):
    names = ['items', 'unit', 'price']
    
    wb = xlrd.open_workbook(file_name)
    # obtain all the sheet in the workbook
    
    sheets_name = wb.sheet_names()
    # print(sheets)
    
    # too less info, omit
    if len(sheets_name) <= 5:
        return
 
    # iterator sheets without the total sheet
    df_all = DataFrame()
    for i in range(len(sheets_name)-1):
    # skiprows=2 omit tow
        df = pd.read_excel(file_name, names = names, sheet_name=i, skiprows=2, 
                       skipfooter = 3, usecols= [0,1,3])
        df_all= df_all.append(df)
        
    
    # print(df_all)

    # delete Nan value and reset index
    df_all = df_all.dropna()
    df_all = df_all.reset_index(drop=True)
    
    # print(df_all)
    
    df_total = pd.read_excel(file_name, 
                             # usecols: use which cols as input
                             # skiprows = 1, jump title and set header to None
                   sheet_name=len(sheets_name)-1,skiprows=1, header = None, names = names, usecols = "A:C")
    # print(df_total)

    # contact active and old database together
    frames = [df_all, df_total]
    results = pd.concat(frames)
    
    # be used for debug
    print(results)
    
    # according to prices, sort value
    results.sort_values(axis=0, ascending=False, inplace=True, 
           kind='quicksort', by=['price'])
    # print(results)
    #based on item and unit to dropout duplicated vlaue
    results.drop_duplicates(['items', 'unit'], keep = 'first', inplace = True)
    
    # reset index
    results = results.reset_index(drop=True)
    
    # print(results)
    
    workbook = openpyxl.load_workbook(file_name)
    # get the name of all workbook
#     name_list = workbook.sheet_names()
    # print(name_list)

    # leave 2,3,4 page as template
    for step, i in enumerate(sheets_name):
        if step == 0 or step == len(sheets_name)-3 or step == 1:
            continue
        
#         print(i)

        # delete workbook with sheet name
        del workbook[i]

    # print(workbook.get_sheet_names())
    workbook._active_sheet_index = 0
    workbook.save(file_name)
    
    # save new data into original excel sheet
    with pd.ExcelWriter(file_name, engine="openpyxl", mode = 'a') as writer:
        # write results to that file_name, name this name in: sheet_name
        # without index
        results.to_excel(writer, sheet_name = sheets_name[len(sheets_name)-1], header = ['品名', '单位', '单价'], index = False)
        writer.save()


In [23]:
if __name__ == '__main__':

    static_dir = "/Users/user/processing_excel/2021.8/"  
    
    any_unrecorder = True
    dir = static_dir
    
    while any_unrecorder:
  
        filename_excel = []
        folder_name = []
        any_unrecorder = False

        for root, dirs, files in os.walk(dir):
            for file in files:

                if not file.endswith('.xlsx'):
                    continue
                # print(os.path.join(root,file))
                filename_excel.append(os.path.join(root,file))

        # print(filename_excel)

        for file in filename_excel:
            print("I m processing ", file)
            load_data(file)
            
        if len(folder_name) != 0:
            temp_folder_name = folder_name.pop(0)
            dir = static_dir + str(temp_folder_name)
            any_unrecorder = True
            
    print("processing finished")

I m processing  /Users/user/processing_excel/2021.8/佳木斯司凡雨.xlsx
I m processing  /Users/user/processing_excel/2021.8/老菜馆.xlsx
I m processing  /Users/user/processing_excel/2021.8/曲线街41号.xlsx
I m processing  /Users/user/processing_excel/2021.8/120.xlsx
I m processing  /Users/user/processing_excel/2021.8/白玛拉姆.xlsx
I m processing  /Users/user/processing_excel/2021.8/二楼海鲜.xlsx
I m processing  /Users/user/processing_excel/2021.8/刘才干调.xlsx
I m processing  /Users/user/processing_excel/2021.8/歌剧院.xlsx
I m processing  /Users/user/processing_excel/2021.8/赵仁伟新232车.xlsx
I m processing  /Users/user/processing_excel/2021.8/广西南宁.xlsx
I m processing  /Users/user/processing_excel/2021.8/百姓人家.xlsx
I m processing  /Users/user/processing_excel/2021.8/水灵灵超市.xlsx
I m processing  /Users/user/processing_excel/2021.8/安埠街441号.xlsx
I m processing  /Users/user/processing_excel/2021.8/建国街279号.xlsx
I m processing  /Users/user/processing_excel/2021.8/大芥末.xlsx
I m processing  /Users/user/processing_excel/2021.8/去茶去.xls

I m processing  /Users/user/processing_excel/2021.8/高频报货/6240.xlsx
        items unit  price
0           盐    件   65.0
1         辣妹子    桶    5.0
2      李锦记香辣酱    瓶   11.0
3      李锦记香辣酱    件  125.0
4         辣妹子    件   84.0
5         瓶东古    件   85.0
6          红糖    袋    4.0
7       户户辣椒粉    袋   17.0
8         川椒段    斤   15.0
9         糯米粉    袋    5.0
10         生抽    件   78.0
11      甘竹玉米粒    件   94.0
12         火碱    斤    4.0
13        糯米粉    袋    5.0
14       劲霸辣根    包   45.0
15         水塔    件   50.0
16        辣妹子    桶    4.0
17      美乐香辣酱    瓶    8.0
18     李锦记香辣酱    瓶   11.0
19        番茄酱    件   95.0
20      如意水晶粉    件   75.0
21         香叶    斤   26.0
22         东古    件  115.0
23       鲜味生抽    件   82.0
24         大料    斤   40.0
25         桂皮    斤   15.0
26         老抽    件   96.0
27         橙汁    瓶   18.0
28       海鲜酱油    件  108.0
29        瓶东古    件   85.0
30     千山红番茄酱    袋    8.0
31    红九九火锅底料    袋   11.0
32        香其酱    件   73.0
33         奶油    块    2.0
34         橙汁    瓶   18

I m processing  /Users/user/processing_excel/2021.8/高频报货/571/南兴街.xlsx
         items unit  price
0          花生脆    袋   30.0
1          川椒段    斤   12.0
2          桂花酱    瓶    5.0
3          红腰豆    瓶    3.0
4           精盐    袋    1.7
5          白砂糖    斤    4.0
6          熟芝麻    斤    9.0
7          孜然脆    斤   12.0
8        丘比沙拉汁    瓶   40.0
9        户户辣椒片    袋   18.0
10          味精    斤    5.0
11         麻辣油    桶   55.0
12         鲜花椒    袋   10.0
13         玉米粒    盒    3.0
14        水塔陈醋    件   48.0
15          炼乳    瓶    7.0
16         川椒段    斤   12.0
17          白糖    斤    3.2
18          白糖    斤    3.2
19       户户辣椒片    袋   18.0
20         树椒段    斤   12.0
21          水塔    件   48.0
22         豆瓣酱    袋    3.0
23           盐    袋    1.0
24          炼乳    桶    7.0
25          白糖    斤    3.2
26         树椒段    斤   12.0
27         树椒段    斤   12.0
28         圆泡椒    件   60.0
29         麻椒粒    斤   40.0
30        丹喜醋精    瓶    5.0
31          东古    桶   58.0
32         树椒段    斤   12.0
33         树

I m processing  /Users/user/processing_excel/2021.8/高频报货/571/西九道.xlsx
I m processing  /Users/user/processing_excel/2021.8/高频报货/571/通江火锅.xlsx
I m processing  /Users/user/processing_excel/2021.8/高频报货/571/通江烧烤.xlsx
I m processing  /Users/user/processing_excel/2021.8/高频报货/571/湘江.xlsx
         items unit  price
0          鲜花椒    袋   10.0
1          辣椒粉    斤   10.0
2           炼乳    桶    7.0
3          辣椒粉    斤   10.0
4           东古    桶   58.0
5           花椒    斤   45.0
6       统厨蒜蓉辣酱    袋    1.0
7          鲜花椒    袋   10.0
8       升园辣白菜酱    袋    7.0
9           白糖    斤    3.2
10          炼乳    桶    7.0
11          炼乳    桶    7.0
12       30度醋精    瓶    7.0
13      升园辣白菜料    袋    7.0
14       佐香园熟酱    袋    5.0
15          火碱    斤    3.0
16           盐    袋    1.0
17          水塔    件   48.0
18          芥梗    斤   25.0
19      升园辣白菜料    袋    7.0
20      升园辣白菜料    袋    7.0
21          芥梗    斤   25.0
22         树椒段    斤   12.0
23         麻辣油    桶   55.0
24         芥末酱    袋   22.0
25         树椒段   

processing finished
